# Análisis exploratorio de datos

## Importación de librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re
import os
import plotly.express as pex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import folium  
from   folium.plugins import HeatMap

## Preliminar

In [ ]:

path_file = r'C:\Users\danie\OneDrive\Documents\Bootcamp_Proyecto\stations.parquet' 
Sta = pd.read_parquet(path_file)
Sta
path_file = r'C:\Users\danie\OneDrive\Documents\Bootcamp_Proyecto\rain.parquet' 
Rain = pd.read_parquet(path_file)
Rain = Rain.sort_values(['Sta','Date'])
Rain


In [ ]:
#Obtener un df donde cada columna muestre la info de los deptos

fecha_columna = 'FechaObservacion' 

In [ ]:
for file in files:
    file_path = os.path.join(folder_path, file)

    df = pd.read_parquet(file_path)
    
    rows, cols = df.shape
    
    if fecha_columna in df.columns:
       
        df[fecha_columna] = pd.to_datetime(df[fecha_columna], errors='coerce')

       
        fecha_minima = df[fecha_columna].min()
        fecha_maxima = df[fecha_columna].max()
        
        print(f"Archivo: {file} - Filas: {rows}, Columnas: {cols}")
        print(f"Fecha mínima: {fecha_minima}")
        print(f"Fecha máxima: {fecha_maxima}\n")
    else:
        print(f"Archivo: {file} - No se encontró la columna '{fecha_columna}'\n")

dataframes = {}


In [ ]:

for file in files:
    file_path = os.path.join(folder_path, file)
    
    df = pd.read_parquet(file_path)
    
    nombre_ciudad = file.split('_')[0]  # Extraer el nombre de la ciudad del nombre del archivo
    dataframes[nombre_ciudad] = df
dataframes={}


In [ ]:
resultados_agrupados = {}
for file in files:
    file_path = os.path.join(folder_path, file)
    
    df = pd.read_parquet(file_path)
    
    df['FechaObservacion'] = pd.to_datetime(df['FechaObservacion'], errors='coerce')
    
    
    df['Year'] = df['FechaObservacion'].dt.year
    df['Month'] = df['FechaObservacion'].dt.month
    
    
    df_grouped = df.groupby(['Year', 'Month','Departamento'], observed=False)['ValorObservado'].agg(['sum', 'mean']).reset_index()
    
    df_grouped.columns = ['Año', 'Mes', 'Departamento','Valor Total', 'Valor Promedio']
    
    # Guardar los resultados en el diccionario
    resultados_agrupados[nombre_ciudad] = df_grouped


In [ ]:
# Llamar al DataFrame de Bogotá
df_bogota =  resultados_agrupados[nombre_ciudad]
df_bogota

In [ ]:
### 1

df_part1 = pd.DataFrame()
for i in namesFiles:
    path_file = path + '\\' + i
    df_dept2 = pd.read_parquet(
                                path_file
                            )
    df_part2 = pd.concat([df_part2,df_dept2], ignore_index=True)
    del df_dept2

df_part1

In [ ]:
### EDA Bogota2
file_bogota=path+r'\rain_bogota2.parquet'
df_bogota = pd.read_parquet(file_bogota)
df_bogota.describe()
df_bogota.describe(include=['category'])

print(df_bogota['Departamento'].value_counts())
print(df_bogota['NombreEstacion'].value_counts())

In [ ]:

df_bogota['Year'] = df_bogota['FechaObservacion'].dt.year
df_bogota['Month'] = df_bogota['FechaObservacion'].dt.month

In [ ]:
year_total = (
    df_bogota
    .groupby(["Year"])["ValorObservado"]
    .sum()
    .reset_index()
)
year_total
year_mean = (
    df_bogota
    .groupby(["Year"])["ValorObservado"]
    .mean()
    .reset_index()
)
year_mean

In [ ]:
fig = make_subplots(rows=1, cols=1
                    #  , row_heights =[0.7, 0.3]
                    # , shared_xaxes = True
                    ,subplot_titles = ['Año'])
fig.add_trace(
    go.Scatter(x=year_total["Year"], y = year_total["ValorObservado"]
               , mode='lines+markers' # selcciona una combinación entre líneas y puntos
               , name = "Observado"
               ,legendgroup = '1'
               ),
)

In [ ]:
month_total = (
    df_bogota
    .groupby(["Month"])["ValorObservado"]
    .sum()
    .reset_index()
)
month_total
fig = make_subplots(rows=1, cols=1
                    #  , row_heights =[0.7, 0.3]
                    # , shared_xaxes = True
                    ,subplot_titles = ['Meses'])
fig.add_trace(
    go.Scatter(x=month_total["Month"], y = month_total["ValorObservado"]
               , mode='lines+markers' # selcciona una combinación entre líneas y puntos
               , name = "Observado"
               ,legendgroup = '1'
               ),
)


In [ ]:
df_bogota['Year'] = df_bogota['FechaObservacion'].dt.year
df_bogota['Month'] = df_bogota['FechaObservacion'].dt.month
df_grouped = df_bogota.groupby(['Year', 'Month'])['ValorObservado'].agg(['sum', 'mean']).reset_index()

df_grouped.columns = ['Year', 'Month', 'Valor Total', 'Valor Promedio']

df_grouped
df_grouped.max()


In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='Month', y='Valor Total', data=df_grouped, marker='o')
plt.title('Distribución del Valor Total Observado por Mes')
plt.xlabel('Mes')
plt.ylabel('Valor Total')
plt.show()
plt.figure(figsize=(12, 6))
sns.lineplot(x='Month', y='Valor Promedio', data=df_grouped, marker='o')
plt.title('Distribución del Valor Promedio Observado por Mes')
plt.xlabel('Mes')
plt.ylabel('Valor Promedio')
plt.show()
plt.figure(figsize=(12, 6))
sns.barplot(x='Year', y='Valor Total', data=df_grouped, palette='Blues_d')
plt.title('Distribución del Valor Total Observado por Año')
plt.xlabel('Año')
plt.ylabel('Valor Total')
plt.show()
plt.figure(figsize=(12, 6))
sns.barplot(x='Year', y='Valor Promedio', data=df_grouped)
plt.title('Distribución del Valor Promedio Observado por Año')
plt.xlabel('Año')
plt.ylabel('Valor Promedio')
plt.show()


In [ ]:
df_pivot = df_grouped.pivot(index='Year', columns='Month', values='Valor Total')


In [ ]:

# Heatmap para visualizar el valor total observado por año y mes
plt.figure(figsize=(12, 6))
sns.heatmap(df_pivot, cmap='coolwarm',annot=True,fmt='.0f')
plt.title('Distribución del Valor Total Observado por Año y Mes')
plt.show()


In [ ]:
df_grouped = df_bogota.groupby(['Year', 'Month','Latitud','Longitud'])['ValorObservado'].agg(['sum', 'mean']).reset_index()

df_grouped.columns = ['Year', 'Month','Latitud','Longitud', 'Valor Total', 'Valor Promedio']

df_grouped
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Latitud', y='Longitud', hue='Valor Total', data=df_grouped, palette='coolwarm')
plt.title('Distribución geográfica de Valor Observado')
plt.show

### 23 últimos

Cantidad de estaciones
X = Rain.loc[Rain['Sta']==0, 'Date'].values
Y = Rain.loc[Rain['Sta']==0, 'Rain'].values

plt.plot(X,Y)
plt.show()

## Exploratory Analysis

In [ ]:
#path_file=r"daniel...\Data\CleanDatasets3 - Merged"

In [2]:
path_file=r"C:\Users\Josue Florez\Documents\Maria Angelica\Proyectos\Proyecto_bootcamp\Data\CleanDatasets3 - Merged\stations.parquet"

Empezaremos a analizar la base de datos de las Estaciones, observaremos su ubicación, cantidad de datos almacenados, entre otras cosas

In [3]:
df_stations=pd.read_parquet(path_file)

In [4]:
df_stations

,Sta,CodSta,NameSta,Lat,Long,Dept,City
0,0,11017020,PR CHOCO: BAGADO,5.412,-76.418,CHOCÓ,BAGADÓ
1,1,11025501,CARMEN DE ATRATO - AUT,5.889,-76.145,CHOCÓ,EL CARMEN
2,2,11027030,EL SIETE - AUT,5.862,-76.152,CHOCÓ,EL CARMEN
3,3,11027070,BORAUDO,5.515,-76.576,CHOCÓ,LLORÓ
4,4,11030010,PR CHOCO: CERTEGUI,5.377,-76.612,CHOCÓ,CÉRTEGUI
...,...,...,...,...,...,...,...
889,889,5205500123,LAS IGLESIAS - AUT,1.055,-77.270,NARIÑO,PASTO
890,890,5311500056,UNIVERSIDAD DEL PACIFICO - AUT,3.848,-76.987,VALLE DEL CAUCA,BUENAVENTURA
891,891,5311500121,LA CUMBRE - AUT,3.645,-76.565,VALLE DEL CAUCA,LA CUMBRE
892,892,5311500149,COLEGIO VASCO NUÑEZ DE BALBOA,3.884,-77.049,VALLE DEL CAUCA,BUENAVENTURA


In [5]:
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894 entries, 0 to 893
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   Sta      894 non-null    int64   
 1   CodSta   894 non-null    uint64  
 2   NameSta  894 non-null    object  
 3   Lat      894 non-null    float64 
 4   Long     894 non-null    float64 
 5   Dept     894 non-null    category
 6   City     894 non-null    category
dtypes: category(2), float64(2), int64(1), object(1), uint64(1)
memory usage: 59.0+ KB


In [6]:
df_stations.isna().sum()

Sta        0
CodSta     0
NameSta    0
Lat        0
Long       0
Dept       0
City       0
dtype: int64

In [8]:
df_stations[df_stations.duplicated(keep=False)]

,Sta,CodSta,NameSta,Lat,Long,Dept,City


In [9]:
df_stations.describe()

,Sta,CodSta,Lat,Long
count,894.000000,8.940000e+02,894.000000,894.000000
mean,446.500000,5.007001e+08,5.505340,-74.636907
std,258.219868,1.048769e+09,2.568901,2.973453
min,0.000000,1.101702e+07,-4.194000,-81.703000
25%,223.250000,2.121517e+07,4.268750,-75.765000
50%,446.500000,2.612702e+07,5.088000,-74.743500
75%,669.750000,5.201702e+07,6.790500,-73.636500
max,893.000000,5.311700e+09,15.797000,0.000000


In [33]:

frecuencia_city=df_stations["City"].value_counts().reset_index()
frecuencia_city.columns=['Ciudad','Cnt']
frecuencia_city_20=frecuencia_city.head(10)
print(frecuencia_city_20)

         Ciudad  Cnt
0  BOGOTÁ, D.C.   76
1   SANTA MARTA    9
2    VILLAMARIA    8
3     AQUITANIA    8
4     MANIZALES    7
5        IBAGUÉ    7
6       PEREIRA    7
7      MONTERÍA    6
8         PASTO    6
9  BUENAVENTURA    6


In [34]:
color_barras='#0083ff'
fig_ciudad = pex.bar(
        frecuencia_city_20,
        x='Ciudad',
        y='Cnt',
        title='Top 20 Ciudades con Mayor Número de Estaciones'
    )
fig_ciudad.update_traces(
            marker_color=color_barras,
            hovertemplate="<b>%{x}</b><br>" +
                         "Cantidad de Estaciones: %{y}<br>" +
                         "<extra></extra>"
        )
fig_ciudad.update_layout(
            plot_bgcolor='white',
            yaxis_title='Cantidad de Estaciones',
            xaxis_tickangle=45,
            height=600,
            showlegend=False,
            hoverlabel=dict(
                bgcolor="white",
                font_size=12))

fig_ciudad

In [37]:
dept_freq = df_stations['Dept'].value_counts().reset_index()
dept_freq.columns = ['Departamento', 'Cantidad']
dept_freq

,Departamento,Cantidad
0,CUNDINAMARCA,84
1,"BOGOTÁ, D.C.",76
2,BOYACÁ,71
3,ANTIOQUIA,66
4,TOLIMA,53
5,HUILA,49
6,SANTANDER,48
7,NORTE DE SANTANDER,38
8,CALDAS,37
9,VALLE DEL CAUCA,36


In [40]:
fig_dept = pex.bar(
        dept_freq,
        x='Departamento',
        y='Cantidad',
        title='Número de Estaciones por Departamento',
        color='Cantidad',
        color_continuous_scale='blues'
    )

    # Personalizar gráfico de departamentos
fig_dept.update_layout(
        plot_bgcolor='white',
        xaxis_title='Departamento',
        yaxis_title='Número de Estaciones',
        xaxis_tickangle=45,
        height=600,
        showlegend=False,
        hoverlabel=dict(
            bgcolor="white",
            font_size=12
        )
    )


In [41]:
folium_map = folium.Map(location=[4.788056, -72.460000], zoom_start=6,tiles="Cartodb dark_matter")
for i in range(0,893):
    marker = folium.CircleMarker(location=[df_stations["Lat"][i],df_stations["Long"][i]],radius=3,
    color="cornflowerblue",
    stroke=False,
    fill=True,
    fill_opacity=0.6,
    opacity=1,
    popup="{}".format(df_stations["NameSta"][i]))
    marker.add_to(folium_map)
folium_map

In [47]:
# Seleccionar solo columnas numéricas
numeric_cols = df_stations.select_dtypes(include=['int64', 'float64', 'uint64']).columns
correlation_matrix = df_stations[numeric_cols].corr()
    
#DataFrame con los valores de correlación 
corr_data = correlation_matrix.reset_index().melt(
        id_vars='index',
        var_name='Variable 2',
        value_name='Correlación'
    )
corr_data.columns = ['Variable 1', 'Variable 2', 'Correlación']
    
# Crear el mapa de calor
fig = pex.imshow(
        correlation_matrix,
        color_continuous_scale='RdBu_r',  # Escala de colores rojo-azul
        aspect='auto',
        title='Matriz de Correlación de Variables Numéricas'
    )
    
# Personalizar el diseño
fig.update_layout(
        width=700,
        height=600,
        xaxis_title='',
        yaxis_title='',
        xaxis={'side': 'bottom'},
        coloraxis_colorbar_title='Correlación'
    )
    
# Añadir los valores numéricos en cada celda
fig.update_traces(
        text=correlation_matrix.round(2),
        texttemplate='%{text}',
        textfont={'size': 12},
        hoverongaps=False,
        hovertemplate='Variable 1: %{y}<br>Variable 2: %{x}<br>Correlación: %{z:.3f}<extra></extra>'
    )
fig

In [42]:
path_file=r"C:\Users\Josue Florez\Documents\Maria Angelica\Proyectos\Proyecto_bootcamp\Data\CleanDatasets3 - Merged\rain0_version2.parquet"

In [43]:
df_rain=pd.read_parquet(path_file)

In [44]:
df_rain.head()

,Sta,CodSen,Date,Rain,Date_diff
0,0,0,2018-05-04 12:00:00,0,NaT
1,0,0,2018-05-04 12:10:00,0,0 days 00:10:00
2,0,0,2018-05-04 12:20:00,0,0 days 00:10:00
3,0,0,2018-05-04 12:30:00,0,0 days 00:10:00
4,0,0,2018-05-04 12:40:00,0,0 days 00:10:00


In [48]:
df_rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190977005 entries, 0 to 190977004
Data columns (total 5 columns):
 #   Column     Dtype          
---  ------     -----          
 0   Sta        uint16         
 1   CodSen     uint8          
 2   Date       datetime64[ns] 
 3   Rain       uint16         
 4   Date_diff  timedelta64[ns]
dtypes: datetime64[ns](1), timedelta64[ns](1), uint16(2), uint8(1)
memory usage: 3.7 GB


In [49]:
df_rain.isna().sum()

Sta            0
CodSen         0
Date           0
Rain           0
Date_diff    894
dtype: int64

Date_diff tiene 894 vacios por cada estacion

In [50]:
df_rain[df_rain.duplicated(keep=False)]

,Sta,CodSen,Date,Rain,Date_diff


In [51]:
df_rain.describe()

,Sta,CodSen,Date,Rain,Date_diff
count,1.909770e+08,190977005.0,190977005,1.909770e+08,190976111
mean,3.835141e+02,0.0,2018-06-06 12:11:48.370322176,6.598080e+01,0 days 00:14:07.431221117
min,0.000000e+00,0.0,2003-01-20 15:20:00,0.000000e+00,0 days 00:00:01
25%,1.810000e+02,0.0,2017-02-14 13:30:00,0.000000e+00,0 days 00:05:00
50%,3.540000e+02,0.0,2019-05-28 16:30:00,0.000000e+00,0 days 00:10:00
75%,5.740000e+02,0.0,2021-02-28 14:15:00,0.000000e+00,0 days 00:10:00
max,8.930000e+02,0.0,2024-10-15 23:59:00,3.000000e+04,3550 days 12:40:00
std,2.453205e+02,0.0,NaN,9.141280e+02,1 days 10:08:05.396294843


In [52]:
df_corr = df_rain.copy()
    
    # Convertir Date_diff a días (número flotante)
df_corr['Date_diff_days'] = df_corr['Date_diff'].dt.total_seconds() / (24 * 60 * 60)
    
    # Seleccionar solo columnas numéricas para la correlación
numeric_cols = ['Sta', 'CodSen', 'Rain', 'Date_diff_days']
correlation_matrix = df_corr[numeric_cols].corr()
    
    # Crear el mapa de calor interactivo
fig = pex.imshow(
        correlation_matrix,
        color_continuous_scale='RdBu_r',  # Escala de colores rojo-azul
        aspect='auto',
        title='Matriz de Correlación - Datos de Lluvia'
    )
    
    # Personalizar el diseño
fig.update_layout(
        width=700,
        height=600,
        xaxis_title='',
        yaxis_title='',
        xaxis={'side': 'bottom'},
        coloraxis_colorbar_title='Correlación'
    )
    
    # Añadir los valores numéricos en cada celda
fig.update_traces(
        text=correlation_matrix.round(3),
        texttemplate='%{text}',
        textfont={'size': 12},
        hoverongaps=False,
        hovertemplate=(
            'Variable 1: %{y}<br>' +
            'Variable 2: %{x}<br>' +
            'Correlación: %{z:.3f}<br>' +
            '<extra></extra>'
        )
    )
fig
    

In [54]:
stats_por_estacion = df_rain.groupby('Sta').agg({
        'Rain': ['sum', 'mean', 'max', 'count']
    }).reset_index()
    
    # Aplanar los nombres de las columnas
stats_por_estacion.columns = ['Estacion', 'Lluvia_Total', 'Lluvia_Promedio', 'Lluvia_Maxima', 'Num_Registros']
    
    # Ordenar por lluvia total descendente
stats_por_estacion = stats_por_estacion.sort_values('Lluvia_Total', ascending=False)

    # Crear figura con subplots
fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Lluvia Total por Estación', 'Lluvia Promedio por Estación'),
        vertical_spacing=0.2
    )

    # Gráfico de barras para lluvia total
fig.add_trace(
        go.Bar(
            x=stats_por_estacion['Estacion'],
            y=stats_por_estacion['Lluvia_Total'],
            name='Lluvia Total',
            hovertemplate="<b>Estación:</b> %{x}<br>" +
                         "<b>Lluvia Total:</b> %{y:.1f}<br>" +
                         "<b>Registros:</b> %{customdata[0]}<br>" +
                         "<b>Máximo:</b> %{customdata[1]:.1f}<br>" +
                         "<extra></extra>",
            customdata=stats_por_estacion[['Num_Registros', 'Lluvia_Maxima']].values,
            marker_color='#1f77b4'
        ),
        row=1, col=1
    )

    # Gráfico de barras para lluvia promedio
fig.add_trace(
        go.Bar(
            x=stats_por_estacion['Estacion'],
            y=stats_por_estacion['Lluvia_Promedio'],
            name='Lluvia Promedio',
            hovertemplate="<b>Estación:</b> %{x}<br>" +
                         "<b>Lluvia Promedio:</b> %{y:.1f}<br>" +
                         "<b>Registros:</b> %{customdata[0]}<br>" +
                         "<b>Máximo:</b> %{customdata[1]:.1f}<br>" +
                         "<extra></extra>",
            customdata=stats_por_estacion[['Num_Registros', 'Lluvia_Maxima']].values,
            marker_color='#2ecc71'
        ),
        row=2, col=1
    )

    # Actualizar diseño
fig.update_layout(
        height=800,
        showlegend=False,
        title_text="Análisis de Lluvia por Estación",
        hovermode='x unified'
    )

    # Actualizar ejes
fig.update_xaxes(title_text="Estación", tickangle=45, row=1, col=1)
fig.update_xaxes(title_text="Estación", tickangle=45, row=2, col=1)
fig.update_yaxes(title_text="Lluvia Total", row=1, col=1)
fig.update_yaxes(title_text="Lluvia Promedio", row=2, col=1)

fig


In [ ]:
fig = pex.bar(df_grouped, x='Sta', y='Valor Total', title='Cantidad de Lluvia por Estación',
             labels={'Sta': 'Estación', 'Valor Total': 'Cantidad de Lluvia'})
fig.show()
##Departamento, filtro

In [ ]:
#Linea de tiempo para cada estacion

In [67]:
# df_rain['Date'] = pd.to_datetime(df_rain['Date'])
    
#     # Ordenar los datos por fecha
# df = df_rain.sort_values('Date')
    
#     # Obtener lista de estaciones únicas
# estaciones = sorted(df['Sta'].unique())
    
#     # Crear figura base
# fig = go.Figure()
    
#     # Añadir una línea por cada estación
# for estacion in estaciones:
#         datos_estacion = df[df['Sta'] == estacion]
#         fig.add_trace(
#             go.Scatter(
#                 x=datos_estacion['Date'],
#                 y=datos_estacion['Rain'],
#                 name=f'Estación {estacion}',
#                 visible=False,  # Inicialmente todas ocultas excepto la primera
#                 hovertemplate=(
#                     "<b>Estación:</b> %{customdata[0]}<br>" +
#                     "<b>Fecha:</b> %{x}<br>" +
#                     "<b>Lluvia:</b> %{y:.1f}<br>" +
#                     "<extra></extra>"
#                 ),
#                 customdata=[[estacion] for _ in range(len(datos_estacion))]
#             )
#         )
    
#     # Hacer visible la primera estación
# if len(estaciones) > 0:
#         fig.data[0].visible = True
    
#     # Crear botones para el selector de estaciones
# def create_buttons():
#         buttons = []
        
#         # Botón para mostrar todas las estaciones
#         buttons.append(
#             dict(
#                 label="Todas las Estaciones",
#                 method="update",
#                 args=[{"visible": [True] * len(estaciones)},
#                      {"title": "Lluvia por Tiempo - Todas las Estaciones"}]
#             )
#         )
        
#         # Botón para cada estación individual
#         for i, estacion in enumerate(estaciones):
#             visible = [False] * len(estaciones)
#             visible[i] = True
#             buttons.append(
#                 dict(
#                     label=f"Estación {estacion}",
#                     method="update",
#                     args=[{"visible": visible},
#                          {"title": f"Lluvia por Tiempo - Estación {estacion}"}]
#                 )
#             )
        
#         return buttons

# # Actualizar el diseño
# fig.update_layout(
#         title="Lluvia por Tiempo",
#         xaxis_title="Fecha",
#         yaxis_title="Lluvia",
#         height=600,
#         showlegend=True,
#         updatemenus=[
#             dict(
#                 type="dropdown",
#                 direction="down",
#                 x=1.0,
#                 y=1.15,
#                 showactive=True,
#                 buttons=create_buttons()
#             )
#         ],
#         # Añadir selector de rango de tiempo
#         xaxis=dict(
#             rangeslider=dict(visible=True),
#             type="date"
#         )
#     )
    
#     # Ajustar márgenes para acomodar el menú desplegable
# fig.update_layout(margin=dict(t=100))

# fig


In [ ]:
# sns.lineplot(data=df_rain, x='Date', y='Rain', hue='Sta', marker="o", palette='tab10')
# plt.xlabel("Fecha")
# plt.ylabel("Cantidad de Lluvia (Rain)")
# plt.title("Cantidad de Lluvia por Estación en el Tiempo")
# plt.show()

In [ ]:
df_anual = df_grouped.groupby(['Year','Valor Total'],observed=True)

df_anual.plot(kind='line', marker='o', color='blue')
plt.xlabel("Año")
plt.ylabel("Valor Total")
plt.title("Año vs Valor Total")
plt.grid()
plt.show()

In [ ]:
# Crear el gráfico interactivo con Plotly
fig = pex.line(df_anual, x='Year', y='Rain', title='Serie Temporal Interactiva de Valor Total',
              labels={'Year': 'Año', 'Rain': 'Valor Total'})
fig.update_traces(mode='lines+markers')  # Agregar puntos sobre la línea
fig.update_layout(hovermode="x")  # Mostrar valores interactivos en el eje x

# Mostrar el gráfico
fig.show()

In [ ]:
fig = pex.line(df_grouped, x='Year', y='Valor Promedio', title='Serie Temporal Interactiva de Valor Total',
              labels={'Year': 'Año', 'Valor Promedio': 'Valor Promedio'})
fig.update_traces(mode='lines+markers')  # Agregar puntos sobre la línea
fig.update_layout(hovermode="x")  # Mostrar valores interactivos en el eje x

# Mostrar el gráfico
fig.show()

In [ ]:
df_anual = df_rain.groupby('Year')['Rain'].mean()

df_anual.plot(kind='line', marker='o', color='blue')
plt.xlabel("Año")
plt.ylabel("Valor Total")
plt.title("Año vs Valor Total")
plt.grid()
plt.show()

In [ ]:
df_anual = df_rain.groupby('Month')['Rain'].sum()

df_anual.plot(kind='line', marker='o', color='blue')
plt.xlabel("Año")
plt.ylabel("Valor Total")
plt.title("Año vs Valor Total")
plt.grid()
plt.show()

In [ ]:
df_anual = df_rain.groupby('Month')['Rain'].mean()

df_anual.plot(kind='line', marker='o', color='blue')
plt.xlabel("Año")
plt.ylabel("Valor Total")
plt.title("Año vs Valor Total")
plt.grid()
plt.show()

In [ ]:
##correlacion
##explicar lo que hago